In [2]:
import random
import tensorflow as tf
import tensorflow.keras.backend as K

import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation, Embedding, Concatenate, Reshape, Lambda
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import OrdinalEncoder

import dice_ml
from dice_ml.utils import helpers # helper functions

from alibi.explainers import CounterFactualProto
from alibi.utils.mapping import ohe_to_ord, ord_to_ohe

import seaborn as sns
import matplotlib.pyplot as plt
import pickle

import time

from xmoai.setup.configure import generate_counterfactuals_classification_proba

In [3]:
seed = 2020
tf.keras.backend.clear_session()
np.random.seed(seed) # NumPy
random.seed(seed) # Python
tf.random.set_seed(seed) # Tensorflow

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

<h5>Loading the dataset, models and metadata</h5>

In [5]:
target_col = 'isFraud'

[X_train_alibi, X_test_alibi, X_test_dice, y_test] = pickle.load(open('validation_dataset.pkl', 'rb'))
[cat_columns, cat_object_columns, cat_vars_ord, numerical_columns, integer_columns, features, enc] = pickle.load(open('encoder_data.pkl', 'rb'))

model_clf = tf.keras.models.load_model('model_clf')
model_clf_proba = tf.keras.models.load_model('model_clf_proba')

<h5>Choosing n individuals (5 true positives and 5 true negatives)</h5>

In [6]:
results_clf = np.round(model_clf.predict(X_test_dice[:3000]), 0).flatten()
results_clf_proba = np.round(model_clf_proba.predict(X_test_alibi[:3000]), 0)[:,1]

In [7]:
# retrieving only the true positives (or negatives) for both models
correct_indexes = np.intersect1d(np.argwhere(results_clf==y_test.values[:3000]).flatten(), np.argwhere(results_clf_proba==y_test.values[:3000]).flatten())

<h5>Results for the validation dataset</h5>

In [8]:
display(confusion_matrix(y_test[:3000], results_clf))
display(confusion_matrix(y_test[:3000], results_clf_proba))

array([[2767,   98],
       [  64,   71]], dtype=int64)

array([[2748,  117],
       [  58,   77]], dtype=int64)

In [9]:
# retrieving n individuals correctly identified for each class
n = 5
X_test_alibi = X_test_alibi.iloc[correct_indexes]
X_test_dice = X_test_dice.iloc[correct_indexes]
y_test = y_test.iloc[correct_indexes]

X_test_alibi = pd.concat([X_test_alibi[y_test==0].head(n), X_test_alibi[y_test==1].head(n)])
X_test_dice = pd.concat([X_test_dice[y_test==0].head(n), X_test_dice[y_test==1].head(n)])
y_test = pd.concat([y_test[y_test==0].head(n), y_test[y_test==1].head(n)])

correct_indexes = X_test_dice.index

In [10]:
X_test_alibi.head()

,TransactionAmt,ProductCD,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9
472433,52.811,0.0,78.0,4.0,4.0,33.0,3.0,332.0,74.0,0.0,0.0,23.0,23.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
472435,136.956,0.0,29.0,73.0,4.0,34.0,3.0,332.0,74.0,0.0,0.0,20.0,20.0,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
472436,25.000,1.0,220.0,42.0,4.0,108.0,3.0,162.0,65.0,0.0,0.0,36.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
472437,23.362,0.0,444.0,73.0,4.0,33.0,2.0,332.0,74.0,0.0,17.0,20.0,20.0,3.0,16.0,0.0,1.0,0.0,1.0,1.0,2.0,0.0,2.0,2.0,2.0,9.0,2.0,53.0,35.0,12.0,82.0,12.0,82.0,12.0,75.125000,0.125000,0.0,0.0,82.0,74.0,82.0,82.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
472438,326.290,4.0,454.0,42.0,4.0,108.0,3.0,14.0,65.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,29.0,29.0,29.0,0.0,0.0,0.0,0.0,0.000000,0.000000,29.0,29.0,0.0,0.0,0.0,29.0,2.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0


<h5>Hyperparameters and Individual handling</h5>
<p>Here we will select one (or more) individuals. We will use the feature dictionary built beforehand in order to finetune the allowed search space (i.e. how much the counterfactual generation algorithms will be able to search).</p>

In [11]:
total_cfs_dice = 4
learning_rates_dice = [0.01, 0.05, 0.1, 0.5, 1]
proximity_weights = [0.05, 0.1, 0.5, 1, 5, 10] 

k_alibi = [None] + list(range(2, 31))

<p>From the data description provided and considering real-world scenarios we are considering the all columns as immutable with the exception of the D* columns and the TransactionAmt. In other words, we want to generate counterfactuals that say <em>"we understand you want to purchase something with your card. If you could wait a little bit longer or if you could spend a little more (or less) you would have a different outcome"</em>. For solely demonstrational purposes the recipient email domain (R_emaildomain) is also added in order to also demonstrate the usage of categorical columns.</p>

<h4>DiCE</h4>

<p>Loading the DiCE model</p>

In [ ]:
backend = 'TF'+tf.__version__[0]
model_clf = tf.keras.models.load_model('model_clf')

In [ ]:
def get_features_from_instance_for_dice(X_current_dice):
    # DiCE: informing which features can be changed and their allowed ranges
    features_dice = features.copy()
    features_to_vary_dice = []

    for col in features_dice.keys():
        if col == 'TransactionAmt':
            # we want to work in a plausible range - for testing purposes, 50% less or 20% more than the current value spent
            features_dice[col] = [int(round(X_current_dice[col].values[0]*0.5)), int(round(X_current_dice[col].values[0]*1.2))]
            features_to_vary_dice.append(col)
        elif col.startswith('dist'):
            features_dice[col] = [0, round(X_current_dice[col].values[0]+500)]
            features_to_vary_dice.append(col)
        elif col.endswith('_emaildomain') or col.startswith('M') or col == 'card6' or col == 'card2' or col == 'card3' or col == 'card4' or col == 'card5':
            features_to_vary_dice.append(col)
        
    return features_dice, features_to_vary_dice

def generate_dice_counterfactuals(learning_rate, proximity_weight, verbose=False):
    d = dice_ml.Data(features=features_dice, outcome_name=target_col)
    m = dice_ml.Model(model=model_clf, backend=backend)

    try:
    # DiCE explanation instance
        exp = dice_ml.Dice(d, m)

        # Generate counterfactual examples
        dice_exp = exp.generate_counterfactuals(query_instance_dice, total_CFs=total_cfs_dice, desired_class="opposite", \
                                                features_to_vary=features_to_vary_dice, \
                                                verbose=verbose, learning_rate=learning_rate, \
                                                proximity_weight=proximity_weight)

        # Visualize counterfactual explanation
        return dice_exp.final_cfs_df.values
    except:
        return np.array([])

In [ ]:
dice_results = {}

counter = 1
for index in correct_indexes:
    display(f'Individual {counter} of {len(correct_indexes)}')
    counter += 1
    current_cfs = []
    recorded_times = np.array([])
    
    run = 1
    for learning_rate in learning_rates_dice:
        for proximity_weight in proximity_weights:
            display(f'Run {run} of {len(learning_rates_dice)*len(proximity_weights)}')
            X_current_dice = X_test_dice.loc[index:index+1]
            y_current_dice = y_test.loc[index:index+1].values[0]
            df_index = X_current_dice.index[0]

            query_instance_dice = X_current_dice.to_dict('index')[df_index]
            features_dice, features_to_vary_dice = get_features_from_instance_for_dice(X_current_dice)

            start_time = time.time()
            
            cfs = generate_dice_counterfactuals(learning_rate, proximity_weight, False)
            #current_cfs.append(cfs[cfs[:,-1]==y_current_dice])
            current_cfs.append(cfs)
            recorded_times = np.append(recorded_times, time.time() - start_time)
            
            run += 1

    dice_results[index] = (current_cfs, recorded_times)

In [ ]:
pickle.dump(dice_results, open('dice_results_clf.pkl', 'wb'))

<h4>Alibi</h4>

In [12]:
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
session = tf.compat.v1.keras.backend.get_session()

Instructions for updating:
non-resource variables are not supported in the long term


<p>Loading the Alibi model</p>

In [13]:
model_clf_proba = tf.keras.models.load_model('model_clf_proba')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
def get_features_from_instance_for_alibi(query_instance_alibi, columns):
    # Alibi: informing which features can be changed and their allowed ranges
    feature_range_alibi = (query_instance_alibi.copy(), query_instance_alibi.copy())
    for col in features.keys():
        current_index = np.argwhere(columns==col).flatten()[0]
        
        if col == 'TransactionAmt':
            # we want to work in a plausible range - for testing purposes, 50% less or 20% more than the current value spent
            feature_range_alibi[0][0][current_index] = query_instance_alibi[0][current_index]*0.5
            feature_range_alibi[1][0][current_index] = query_instance_alibi[0][current_index]*1.2
        elif col.startswith('dist'):
            feature_range_alibi[0][0][current_index] = 0
            feature_range_alibi[1][0][current_index] = query_instance_alibi[0][current_index]+500
        elif col.endswith('_emaildomain') or col.startswith('M') or col == 'card6' or col == 'card2' or col == 'card3' or col == 'card4' or col == 'card5':
            feature_range_alibi[0][0][current_index] = 0
            feature_range_alibi[1][0][current_index] = len(features[col]) - 1
        
    return feature_range_alibi

In [ ]:
alibi_results = {}

counter = 1
for index in correct_indexes:
    display(f'Individual {counter} of {len(correct_indexes)}')
    counter += 1
    current_cfs = []
    recorded_times = np.array([])
    
    X_current_alibi = X_test_alibi.loc[index:index]
    query_instance_alibi = X_current_alibi.values

    feature_range_alibi = get_features_from_instance_for_alibi(query_instance_alibi, X_current_alibi.columns)
    cf = CounterFactualProto(sess=session,
                             predict=model_clf_proba,
                             shape=(1,) + X_test_alibi.shape[1:],
                             cat_vars=cat_vars_ord,
                             feature_range=feature_range_alibi
                             #learning_rate_init=0.5
                            )
    
    cf.fit(X_train_alibi.values);
    
    run = 1
    for k in k_alibi:
        display(f'Run {run} of {len(k_alibi)}, k={k}')
    
        start_time = time.time()
        explanation = cf.explain(query_instance_alibi, k=k)
        
        try:
            current_cfs.append(explanation['cf']['X'])
            display(explanation)
            display(f'CF found for the individual in {index}')
        except:
            display(f'No CFs found for the individual in {index}')
        
        recorded_times = np.append(recorded_times, time.time() - start_time)

        run += 1

    alibi_results[index] = (current_cfs, recorded_times)

In [ ]:
pickle.dump(alibi_results, open('alibi_results_clf.pkl', 'wb'))

<h5>xMOAI</h5>

In [14]:
def get_features_from_instance_for_xmoai(query_instance_xmoai, columns):
    # xMOAI: informing which features can be changed and their allowed ranges
    immutable_cols = []
    upper_bounds = query_instance_xmoai.copy()
    lower_bounds = query_instance_xmoai.copy()
    categorical_columns_xmoai = {}

    for col in features.keys():
        current_index = np.argwhere(columns==col).flatten()[0]

        if col == 'TransactionAmt':
            # we want to work in a plausible range - for testing purposes, 50% less or 20% more than the current value spent
            lower_bounds[current_index] = query_instance_xmoai[current_index]*0.5
            upper_bounds[current_index] = query_instance_xmoai[current_index]*1.2
        elif col.startswith('dist'):
            lower_bounds[current_index] = 0
            upper_bounds[current_index] = query_instance_xmoai[current_index]+500
        elif col.endswith('_emaildomain') or col.startswith('M') or col == 'card6' or col == 'card2' or col == 'card3' or col == 'card4' or col == 'card5':
            lower_bounds[current_index] = 0
            upper_bounds[current_index] = len(features[col]) - 1
        elif col in cat_columns:
            lower_bounds[current_index] = 0
            upper_bounds[current_index] = len(features[col]) - 1
            categorical_columns_xmoai[current_index] = list(range(len(features[col])))
            immutable_cols.append(current_index)
        else:
            lower_bounds[current_index] = features[col][0]
            upper_bounds[current_index] = features[col][1]
            immutable_cols.append(current_index)
    
    return immutable_cols, upper_bounds, lower_bounds, categorical_columns_xmoai

In [15]:
xmoai_results = {}

counter = 1
for index in correct_indexes:
    display(f'Individual {counter} of {len(correct_indexes)}')
    counter += 1
    current_cfs = []
    current_front = []
    current_algs = []
    recorded_times = np.array([])
    
    X_current_xmoai = X_test_alibi.loc[index:index]
    query_instance_xmoai = X_current_xmoai.values[0]
    
    # get prediction information
    current_prediction = model_clf_proba.predict(np.array([query_instance_xmoai]))
    num_classes = current_prediction.shape[1]

    y_desired = np.argmin(np.round(np.array(current_prediction)))
    y_acceptable_range = np.array([1.0/num_classes, 1.0])
    immutable_cols, upper_bounds, lower_bounds, categorical_columns_xmoai = get_features_from_instance_for_xmoai(query_instance_xmoai, X_current_xmoai.columns)
    max_changed_vars = len(query_instance_xmoai) - len(immutable_cols)

    run = 1
    limit = 30
    for i in list(range(max_changed_vars, 4, -1)) * 2:
        display(f'Run {run} of {limit} (maximum immutable columns: {i})')
    
        start_time = time.time()
        pareto_front, pareto_set, pareto_algorithms = generate_counterfactuals_classification_proba(model_clf_proba, query_instance_xmoai,
                                                                        y_desired, immutable_cols, y_acceptable_range, 
                                                                        upper_bounds, lower_bounds, categorical_columns_xmoai,
                                                                        integer_columns, method_name='predict', max_changed_vars=i,
                                                                        verbose=False, seed=seed, n_jobs=-1)
        
        try:
            current_cfs.append(pareto_set.tolist())
            current_front.append(pareto_front.tolist())
            current_algs.append(pareto_algorithms.tolist())
            display(f'Run finished.')
        except:
            display(f'Run finished with errors.')
        
        recorded_times = np.append(recorded_times, time.time() - start_time)

        run += 1

    xmoai_results[index] = (current_cfs, current_front, current_algs, recorded_times)

'Individual 1 of 10'

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


'Run 1 of 30 (maximum immutable columns: 19)'


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.configuration import Configuration
Configuration.show_compile_hint = False



divide by zero encountered in true_divide
invalid value encountered in true_divide


...

'Run finished.'

'Run 29 of 30 (maximum immutable columns: 6)'

'Run finished.'

'Run 30 of 30 (maximum immutable columns: 5)'

'Run finished.'

In [16]:
pickle.dump(xmoai_results, open('xmoai_results_clf.pkl', 'wb'))